# **Notebook de Preparação de Dados para Projeto de Identificação de Depressão**

**Introdução:**
Neste notebook, iremos preparar os dados para a construção de modelos de machine learning. Os atributos utlizados foram selecionados criteriozamente utilizando como base o método CAPTO para selecionar os atributos mais relevantes para a classificação de pessoas que possam possuir depressão. Os dados utilizados serão provenientes da Pesquisa Nacional de Saúde (PNS) de 2019.

**Objetivos:**
O principal objetivo deste notebook é realizar a preparação inicial dos dados, incluindo o tratamento de valores ausentes e vazios, conversão de tipos de variáveis, junção de atributos e discretizarção dos dados, além da análise exploratória dos dados filtrados.


## Bibliotecas Utilizadas
As principais bibliotecas utilizadas nesta etapa são:
- Sidetable: Auxilia na identificação rápida de valores faltantes e frequências de valores categóricos.
- Pandas Profiling: Faz uma análise exploratória automática dos dados, gerando insights sobre problemas nos dados.
- NumPy: Biblioteca fundamental para computação científica em Python.
- Pandas: Biblioteca popular para análise de dados.
- Seaborn / Matplotlib: Visualização de dados.
- Missingno: Facilita a plotagem de dados faltantes.
- IPython Widgets: Permite interatividade.

In [44]:
import numpy as np
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt 

import sidetable 
from ydata_profiling import ProfileReport

import missingno as msno 
from ipywidgets import interact, widgets 

from sklearn.preprocessing import scale, minmax_scale, power_transform

In [45]:
sns.set_theme(
    context = 'talk', 
    style='ticks',
    font_scale= 8,
    rc= {
        'figure.figsize': (12,8) 
    }
)

# Análise Inicial da Base de dados

### Importando os dados 
> Os dados filtrados estão em formato .csv 

In [46]:
df = pd.read_csv(r'C:\Users\maype\OneDrive\Área de Trabalho\projects\projeto-aprendizado-de-maquina\Data\linhas_e_colunas_filtradas.csv')


> Visualização inicial dos dados 

In [47]:
df.head()

,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P01601,P018,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601
0,2.0,11.0,1963.0,55.0,3.0,0.0,100.0,1.0,NaN,NaN,...,0.0,3.0,NaN,1.0,2.0,7.0,2.0,0.0,0.0,3.0
1,2.0,11.0,1973.0,45.0,1.0,1.0,1000.0,1.0,NaN,NaN,...,7.0,7.0,2.0,0.0,NaN,0.0,NaN,0.0,0.0,4.0
2,2.0,8.0,1960.0,58.0,1.0,0.0,240.0,1.0,NaN,NaN,...,3.0,3.0,NaN,3.0,2.0,0.0,NaN,0.0,0.0,3.0
3,2.0,99.0,9999.0,41.0,3.0,0.0,998.0,1.0,NaN,NaN,...,7.0,7.0,2.0,1.0,2.0,7.0,1.0,1.0,0.0,3.0
4,2.0,12.0,1966.0,52.0,2.0,1.0,250.0,1.0,NaN,NaN,...,3.0,3.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,3.0


> Vizualizando quantas colunas e linhas temos 

In [48]:
# Obtendo o número de linhas e colunas 
num_linhas, num_colunas = df.shape

print(f'Possuimos {num_linhas} linhas e {num_colunas} colunas.')


Possuimos 40359 linhas e 138 colunas.


> Verificando quantidade de valores nulos 

In [49]:
# Filtrar colunas com valores nulos maiores que zero
colunas_com_nulos = df.columns[df.isna().sum() > 0]

# Iterar sobre essas colunas e imprimir as informações apenas se o número de nulos for maior que zero
for coluna in colunas_com_nulos:
    num_nulos = df[coluna].isna().sum()
    print(f'Coluna "{coluna}" tem {num_nulos} valores nulos.')



Coluna "E01602" tem 14459 valores nulos.
Coluna "E01601" tem 14459 valores nulos.
Coluna "E01603" tem 40174 valores nulos.
Coluna "E01604" tem 40174 valores nulos.
Coluna "E01605" tem 40355 valores nulos.
Coluna "E01801" tem 38923 valores nulos.
Coluna "E01802" tem 38923 valores nulos.
Coluna "E01803" tem 40334 valores nulos.
Coluna "E01804" tem 40334 valores nulos.
Coluna "E01805" tem 40351 valores nulos.
Coluna "F001021" tem 31524 valores nulos.
Coluna "F007021" tem 39261 valores nulos.
Coluna "F008021" tem 38334 valores nulos.
Coluna "VDF00102" tem 35087 valores nulos.
Coluna "E01201" tem 14074 valores nulos.
Coluna "E01401" tem 14074 valores nulos.
Coluna "E014011" tem 40064 valores nulos.
Coluna "E002" tem 23903 valores nulos.
Coluna "E019" tem 38893 valores nulos.
Coluna "E017" tem 14074 valores nulos.
Coluna "M005011" tem 37214 valores nulos.
Coluna "M00601" tem 37214 valores nulos.
Coluna "D00901" tem 3763 valores nulos.
Coluna "Q09201" tem 35765 valores nulos.
Coluna "Q09202" 

> Conferindo tipos de dados 

In [50]:
df.dtypes

C006      float64
C00702    float64
C00703    float64
C008      float64
M01401    float64
           ...   
P023      float64
P02401    float64
P02501    float64
P02602    float64
P02601    float64
Length: 138, dtype: object

>Observamos que todas as variáveis estão sendo considerados **float**, oque não condiz com a realidade. 

> Vamos converte-las para variáveis do tipo **categorico** e manter somente as variaveis que realmente são numéricas como **float**

In [51]:
df[ [
  'C008','C00702', 'C00703',
  'W00101', 'W00102', 'W00201', 'W00202',  # Peso, altura e idade
  'A01001','A011',
  'M01401', 'M01501', 'M005011','P029','P02801',
  'E01602', 'E01601', 'E01605', 'E01802', 'E01804', 'E019', 'E017',
  'F001021', 'F007021', 'F008021', 'VDF00102','P03702','P03701', 'P006', 
  'P00901', 'P01001', 'P01101', 'P013','P015', 'P02001', 'P02101',  'P01601','P018', 'P019',  'P02602',
  'P02002', 'P02102',  'P023', 'P02401', 'P02501', 'P02602', 'P00403', 'P00404', 'P00103', 'P00104'
]]

,C008,C00702,C00703,W00101,W00102,W00201,W00202,A01001,A011,M01401,...,P02002,P02102,P023,P02401,P02501,P02602,P00403,P00404,P00103,P00104
0,55.0,11.0,1963.0,NaN,NaN,NaN,NaN,5.0,2.0,3.0,...,1.0,2.0,7.0,2.0,0.0,0.0,145.0,145.0,60.0,60.0
1,45.0,11.0,1973.0,NaN,NaN,NaN,NaN,8.0,4.0,1.0,...,0.0,NaN,0.0,NaN,0.0,0.0,155.0,155.0,77.0,77.0
2,58.0,8.0,1960.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,3.0,2.0,0.0,NaN,0.0,0.0,NaN,159.0,NaN,56.0
3,41.0,99.0,9999.0,NaN,NaN,NaN,NaN,6.0,2.0,3.0,...,1.0,2.0,7.0,1.0,1.0,0.0,170.0,170.0,75.0,75.0
4,52.0,12.0,1966.0,NaN,NaN,NaN,NaN,2.0,1.0,2.0,...,0.0,NaN,0.0,NaN,0.0,0.0,149.0,149.0,52.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40354,51.0,9.0,1967.0,NaN,NaN,NaN,NaN,5.0,1.0,2.0,...,0.0,NaN,2.0,2.0,0.0,0.0,158.0,158.0,62.0,62.0
40355,44.0,5.0,1975.0,NaN,NaN,NaN,NaN,5.0,3.0,3.0,...,1.0,2.0,1.0,2.0,7.0,0.0,162.0,162.0,62.0,62.0
40356,54.0,10.0,1964.0,NaN,NaN,NaN,NaN,5.0,2.0,3.0,...,1.0,2.0,0.0,NaN,0.0,0.0,168.0,168.0,63.0,63.0
40357,44.0,7.0,1975.0,NaN,NaN,NaN,NaN,5.0,2.0,2.0,...,0.0,NaN,0.0,NaN,0.0,0.0,168.0,168.0,72.0,72.0


In [52]:
# Converter todas as colunas para categóricas
df = df.astype('category')

# Lista das colunas que devem ser numéricas
colunas_numericas = [
    'C008', 'C00702', 'C00703',
    'W00101', 'W00102', 'W00201', 'W00202',  # Peso, altura e idade
    'A01001', 'A011',
    'M01401', 'M01501', 'M005011', 'P029', 'P02801',
    'E01602', 'E01601', 'E01605', 'E01802', 'E01804', 'E019', 'E017',
    'F001021', 'F007021', 'F008021', 'VDF00102', 'P03702', 'P03701', 'P006',
    'P00901', 'P01001', 'P01101', 'P013', 'P015', 'P02001', 'P02101', 'P01601', 'P018', 'P019', 'P02602',
    'P02002', 'P02102', 'P023', 'P02401', 'P02501', 'P02602', 'P00403', 'P00404', 'P00103', 'P00104'
]

# Tentar converter as colunas selecionadas para numérico
for coluna in colunas_numericas:
    try:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
    except ValueError:
        print(f"A coluna {coluna} contém valores não numéricos e não pode ser convertida.")

# Verificar os tipos de dados após a conversão
print(df.dtypes)


C006      category
C00702     float64
C00703     float64
C008       float64
M01401     float64
            ...   
P023       float64
P02401     float64
P02501     float64
P02602     float64
P02601    category
Length: 138, dtype: object


> Agora que nossos tipos de dados estão corretos, iremos gerar um relatório para análise e melhor compreenção de nossos dados. 

In [53]:
# Gerando Relatorio para Análises Gerais
profile = ProfileReport(df, title="Profiling Report")
profile.to_file(r"C:\Users\maype\OneDrive\Área de Trabalho\projects\projeto-mineracao-de-dados\Vizualizaçoes\EDA_pns2019_depressao.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\maype\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
C:\Users\maype\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
C:\Users\maype\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
C:\Users\maype\AppData\Local\Packages\PythonSo

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Tratando dados 


## Dados duplicados
> Verificando se temos valores duplicados 

In [54]:
# Identificando Valores Duplicados
df[df.duplicated(keep=False)]

,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P01601,P018,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601
25251,1.0,1.0,1960.0,59.0,3.0,2.0,12000.0,1.0,NaN,NaN,...,0.0,0.0,NaN,2.0,2.0,7.0,1.0,0.0,0.0,3.0
25253,1.0,1.0,1960.0,59.0,3.0,2.0,12000.0,1.0,NaN,NaN,...,0.0,0.0,NaN,2.0,2.0,7.0,1.0,0.0,0.0,3.0


> Agora que idetificamos duplicatas em nossos dados, vamos remove-las 


In [55]:
#Removendo valores duplicados 
df.drop_duplicates()

,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P01601,P018,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601
0,2.0,11.0,1963.0,55.0,3.0,0.0,100.0,1.0,NaN,NaN,...,0.0,3.0,NaN,1.0,2.0,7.0,2.0,0.0,0.0,3.0
1,2.0,11.0,1973.0,45.0,1.0,1.0,1000.0,1.0,NaN,NaN,...,7.0,7.0,2.0,0.0,NaN,0.0,NaN,0.0,0.0,4.0
2,2.0,8.0,1960.0,58.0,1.0,0.0,240.0,1.0,NaN,NaN,...,3.0,3.0,NaN,3.0,2.0,0.0,NaN,0.0,0.0,3.0
3,2.0,99.0,9999.0,41.0,3.0,0.0,998.0,1.0,NaN,NaN,...,7.0,7.0,2.0,1.0,2.0,7.0,1.0,1.0,0.0,3.0
4,2.0,12.0,1966.0,52.0,2.0,1.0,250.0,1.0,NaN,NaN,...,3.0,3.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40354,2.0,9.0,1967.0,51.0,2.0,0.0,1200.0,1.0,NaN,NaN,...,7.0,7.0,3.0,0.0,NaN,2.0,2.0,0.0,0.0,3.0
40355,1.0,5.0,1975.0,44.0,3.0,0.0,1200.0,1.0,NaN,NaN,...,0.0,2.0,NaN,1.0,2.0,1.0,2.0,7.0,0.0,3.0
40356,2.0,10.0,1964.0,54.0,3.0,3.0,4000.0,1.0,NaN,NaN,...,7.0,2.0,NaN,1.0,2.0,0.0,NaN,0.0,0.0,4.0
40357,1.0,7.0,1975.0,44.0,2.0,2.0,2000.0,1.0,NaN,NaN,...,2.0,3.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,4.0


## Consolidando Colunas 
> Uma forma de tentar reduzir o número de valores ausentes e faltantes em nossa base de dados é consolidar nossos atributos, fazendo várias colunas virarem uma só 

### Consolidando colunas de Peso e Altura

In [56]:

# Preencher os valores nulos nas colunas de peso e altura usando todas as colunas disponíveis
df['peso_consolidado'] = df[['P00102', 'P00103', 'P00104', 'W00101', 'W00102']].fillna(method='ffill', axis=1)['P00104']
df['altura_consolidada'] = df[['P00402', 'P00403', 'P00404', 'W00201', 'W00202']].fillna(method='ffill', axis=1)['P00404']

# Remover as colunas originais de peso e altura
df.drop(columns=['P00102', 'P00103', 'P00104', 'W00101', 'W00102', 'P00402', 'P00403', 'P00404', 'W00201', 'W00202'], inplace=True)


# Exibir as primeiras linhas para verificar se os valores foram preenchidos corretamente e as colunas foram removidas
df.head()


C:\Users\maype\AppData\Local\Temp\ipykernel_8712\1541160845.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['peso_consolidado'] = df[['P00102', 'P00103', 'P00104', 'W00101', 'W00102']].fillna(method='ffill', axis=1)['P00104']
C:\Users\maype\AppData\Local\Temp\ipykernel_8712\1541160845.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['altura_consolidada'] = df[['P00402', 'P00403', 'P00404', 'W00201', 'W00202']].fillna(method='ffill', axis=1)['P00404']


,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601,peso_consolidado,altura_consolidada
0,2.0,11.0,1963.0,55.0,3.0,0.0,100.0,1.0,NaN,NaN,...,NaN,1.0,2.0,7.0,2.0,0.0,0.0,3.0,60.0,145.0
1,2.0,11.0,1973.0,45.0,1.0,1.0,1000.0,1.0,NaN,NaN,...,2.0,0.0,NaN,0.0,NaN,0.0,0.0,4.0,77.0,155.0
2,2.0,8.0,1960.0,58.0,1.0,0.0,240.0,1.0,NaN,NaN,...,NaN,3.0,2.0,0.0,NaN,0.0,0.0,3.0,56.0,159.0
3,2.0,99.0,9999.0,41.0,3.0,0.0,998.0,1.0,NaN,NaN,...,2.0,1.0,2.0,7.0,1.0,1.0,0.0,3.0,75.0,170.0
4,2.0,12.0,1966.0,52.0,2.0,1.0,250.0,1.0,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,3.0,52.0,149.0


In [57]:
# contando valores nulos
nullp = df['peso_consolidado'].isna().sum()
nulla = df['altura_consolidada'].isna().sum()

print(f"ficamos com { nullp } valores nulos em Peso e { nulla } valores nulos em Altura") 

ficamos com 0 valores nulos em Peso e 0 valores nulos em Altura


### Consolidando colunas de Renda

In [58]:
# Definir as colunas relacionadas à renda que serão somadas
colunas_renda = [

    'E01602', 'E01601', 'E01605', 'E01802', 'E01804',
    'F001021', 'F007021', 'F008021', 'VDF00102',
]

# Preencher os valores nulos nas colunas de renda com zero
df[colunas_renda] = df[colunas_renda].fillna(0)

# Somar as colunas relacionadas à renda em uma nova coluna 'renda_total'
df['renda_total'] = df[colunas_renda].sum(axis=1)

# Remover as colunas originais relacionadas à renda
df.drop(columns=colunas_renda, inplace=True)

# Exibir as primeiras linhas para verificar se os valores foram preenchidos corretamente e as colunas foram removidas
df.head()

,C006,C00702,C00703,C008,M01401,M01501,E01603,E01604,E01801,E01803,...,P02002,P02102,P023,P02401,P02501,P02602,P02601,peso_consolidado,altura_consolidada,renda_total
0,2.0,11.0,1963.0,55.0,3.0,0.0,NaN,NaN,NaN,NaN,...,1.0,2.0,7.0,2.0,0.0,0.0,3.0,60.0,145.0,101.0
1,2.0,11.0,1973.0,45.0,1.0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,0.0,4.0,77.0,155.0,1001.0
2,2.0,8.0,1960.0,58.0,1.0,0.0,NaN,NaN,NaN,NaN,...,3.0,2.0,0.0,NaN,0.0,0.0,3.0,56.0,159.0,325.0
3,2.0,99.0,9999.0,41.0,3.0,0.0,NaN,NaN,NaN,NaN,...,1.0,2.0,7.0,1.0,1.0,0.0,3.0,75.0,170.0,999.0
4,2.0,12.0,1966.0,52.0,2.0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,0.0,3.0,52.0,149.0,251.0


In [59]:
# contando valores nulos
df['renda_total'].isna().sum()

0

###  Condolidando Idade, Ano de Nascimento e Mês de Nascimento 


In [60]:
# Calcular a idade apenas para os valores nulos de idade
df['idade'] = np.where(df['C008'].isnull(), 2019 - df['C00703'], df['C008'])

# Remover as colunas de mês e ano de nascimento, e a coluna de idade original
df.drop(columns=['C00702', 'C00703', 'C008'], inplace=True)

# Exibir as primeiras linhas para verificar se o cálculo da idade foi feito corretamente e as colunas foram removidas
print(df['idade'].head())

0    55.0
1    45.0
2    58.0
3    41.0
4    52.0
Name: idade, dtype: float64


In [61]:
# contando valores nulos
df['idade'].isna().sum()

0

### Consolidando colunas com informações sobre o trabalho 

In [62]:
# Verificar se ambas as colunas 'E001' e 'E002' estão nulas e atribuir NaN para 'trabalhou'
df['trabalhou'] = df.apply(lambda row: np.nan if pd.isna(row['E001']) and pd.isna(row['E002']) else row['E001'] if pd.notna(row['E001']) else row['E002'], axis=1)

# Remover as colunas 'E001' e 'E002'
df.drop(columns=['E001', 'E002'], inplace=True)

# Exibir as primeiras linhas para verificar se a junção foi feita corretamente e as colunas foram removidas
print(df['trabalhou'].head)

<bound method NDFrame.head of 0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
40354    1.0
40355    1.0
40356    1.0
40357    1.0
40358    1.0
Name: trabalhou, Length: 40359, dtype: float64>


In [63]:
df['trabalhou'].isna().sum()

0

### Consolidando colunas sobre medicamento para depressão 

In [64]:
# Verificar se ambas as colunas 'Q09201' e 'Q09202' estão nulas e atribuir NaN para 'medicamento_depressao'
df['medicamento_depressao'] = df.apply(lambda row: np.nan if pd.isna(row['Q09201']) and pd.isna(row['Q09202']) else row['Q09201'] if pd.notna(row['Q09201']) else row['Q09202'], axis=1)

# Remover as colunas 'Q09201' e 'Q09202'
df.drop(columns=['Q09201', 'Q09202'], inplace=True)

# Exibir as primeiras linhas para verificar se a junção foi feita corretamente e as colunas foram removidas
df['medicamento_depressao'].head()


0    1.0
1    NaN
2    NaN
3    NaN
4    NaN
Name: medicamento_depressao, dtype: float64

In [65]:
# contando valores nulos
df['medicamento_depressao'].isna().sum()

35765

> Observamos que mesmo consolidando vários atributos, nosso novo atributo **"medicamento_depressao"** ainda contem 35.765 intâncias **nulas**. 
> Vamos optar por remover este atributo por dois motivos: 
>  - Quantidade de valores nulos. 
>  - Relação direta com nosso dominio de problema, oque pode interferir na construção de nosso modelo. 

In [66]:
df.drop(columns=['medicamento_depressao'], inplace=True)

### Consolidando Colunas de Horas Trabalhadas

In [67]:


# Definir as colunas relacionadas às horas trabalhadas que serão somadas
colunas_horas_trabalhadas = ['E017', 'E019']

# Converter colunas categóricas para numéricas
df[colunas_horas_trabalhadas] = df[colunas_horas_trabalhadas].apply(pd.to_numeric, errors='coerce')

# Preencher os valores nulos nas colunas de horas trabalhadas com zero
df[colunas_horas_trabalhadas] = df[colunas_horas_trabalhadas].fillna(0)

# Somar as colunas relacionadas às horas trabalhadas em uma nova coluna 'horas_trabalhadas_total'
df['horas_trabalhadas_total'] = df[colunas_horas_trabalhadas].sum(axis=1)

# Remover as colunas originais relacionadas às horas trabalhadas
df.drop(columns=colunas_horas_trabalhadas, inplace=True)

# Exibir as primeiras linhas para verificar se os valores foram preenchidos corretamente e as colunas foram removidas
df.head()

,C006,M01401,M01501,E01603,E01604,E01801,E01803,E01805,F001011,F007011,...,P02401,P02501,P02602,P02601,peso_consolidado,altura_consolidada,renda_total,idade,trabalhou,horas_trabalhadas_total
0,2.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,...,2.0,0.0,0.0,3.0,60.0,145.0,101.0,55.0,1.0,40.0
1,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,...,NaN,0.0,0.0,4.0,77.0,155.0,1001.0,45.0,1.0,36.0
2,2.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,...,NaN,0.0,0.0,3.0,56.0,159.0,325.0,58.0,1.0,6.0
3,2.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,...,1.0,1.0,0.0,3.0,75.0,170.0,999.0,41.0,1.0,16.0
4,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,...,NaN,0.0,0.0,3.0,52.0,149.0,251.0,52.0,1.0,48.0


In [68]:
# contando valores nulos
df['horas_trabalhadas_total'].isna().sum()

0

### Consolidando colunas sobre doençãs crônicas 

In [69]:
def definir_doencas_cronicas(row):
    if row['J007'] == 1 or row['Q11006'] == 1 or row['Q11007'] == 1 or row['Q11008'] == 1 or row['Q11009'] == 1 or row['Q11010'] == 1:
        return 1  # Possui doença crônica
    else:
        return 2  # Não possui doença crônica

# Criar a coluna 'doencas_cronicas' aplicando a função definir_doencas_cronicas
df['doencas_cronicas'] = df.apply(definir_doencas_cronicas, axis=1)

#Remover as colunas que não serão mais necessárias
colunas_para_remover = ['J007', 'Q11006', 'Q11007', 'Q11008', 'Q11009', 'Q11010']
df.drop(columns=colunas_para_remover, inplace=True)

# Exibir as primeiras linhas para verificar a criação da nova coluna
 
df['doencas_cronicas'].head()


0    1
1    2
2    2
3    2
4    1
Name: doencas_cronicas, dtype: int64

In [70]:
df['doencas_cronicas'].isna().sum()

0

### Consolidando colunas sobre saneamento básico

> Caracterização de Saneamento Básico
>
> - Código
> Nesta célula, foi implementada uma função em Python para caracterizar a situação de saneamento básico com base em métricas específicas.
>
> A função `definir_saneamento()` avalia diferentes condições relacionadas ao saneamento básico, como acesso a água potável, esgotamento sanitário e coleta de lixo.
>
> - Métricas Utilizadas
> As métricas utilizadas foram baseadas em questões específicas de um questionário do IBGE relacionadas ao saneamento básico. Essas métricas foram escolhidas por representarem indicadores importantes de acesso a serviços básicos de infraestrutura.



In [71]:
def definir_saneamento(row):
    # Verificar as condições de saneamento básico
    if row['A005012'] == 1 and row['A00601'] == 1 and row['A009010'] in [1, 3, 4]:
        if row['A01401'] > 0 and row['A01402'] == 1 and row['A01403'] == 1 and row['A01501'] == 1 and row['A016010'] in [1, 2, 4]:
            return 1  # Saneamento básico completo (sim)
        else:
            return 1  # Saneamento básico parcial (sim)
    else:
        return 2  # Sem saneamento básico (não)

# Criar a coluna 'saneamento_basico' aplicando a função definir_saneamento
df['saneamento_basico'] = df.apply(definir_saneamento, axis=1)

# Remover as colunas que não serão mais necessárias
colunas_para_remover = ['A005012', 'A00601', 'A009010', 'A01401', 'A01402', 'A01403', 'A01501', 'A016010']
df.drop(columns=colunas_para_remover, inplace=True)

# Exibir as primeiras linhas para verificar a criação da nova coluna
df['saneamento_basico'].head()


0    2
1    2
2    2
3    2
4    2
Name: saneamento_basico, dtype: int64

In [72]:
# contando valores nulos
df['saneamento_basico'].isna().sum()

0

In [73]:
df['saneamento_basico'].values

array([2, 2, 2, ..., 1, 2, 2], dtype=int64)

### Consolidando colunas sobre moradia

> Caracterização de Moradia Vulnerável
>
> - Código
> 
>Nesta célula, foi implementado um código em Python para caracterizar a situação de moradia vulnerável com > base em métricas fornecidas pelo IBGE.
>
>Foi utilizada uma função que avalia diferentes características do domicílio, como tipo de construção, >materiais utilizados nas paredes, cobertura e piso, além da quantidade de cômodos e dormitórios.
>
> - Métricas do IBGE
> 
>As métricas utilizadas foram baseadas em questões específicas de um questionário do IBGE relacionadas à habitação e condições de moradia. Essas métricas foram escolhidas por representarem indicadores importantes de vulnerabilidade habitacional.


In [74]:
def caracterizar_moradia_vulneravel(row):
    tipo_domicilio = row['A001']
    material_paredes = row['A002010']
    material_cobertura = row['A003010']
    material_piso = row['A004010']
    num_comodos = row['A01001']
    num_dormitorios = row['A011']
    
    # Verificar as condições de moradia vulnerável
    if (tipo_domicilio == 3 or material_paredes in [3, 5] or material_cobertura in [3, 5] or 
        material_piso in [4] or num_comodos < num_dormitorios):
        return '1'  # Moradia vulnerável
    else:
        return '2'  # Moradia não vulnerável

# Criar a coluna 'moradia_vulneravel' aplicando a função caracterizar_moradia_vulneravel
df['moradia_vulneravel'] = df.apply(caracterizar_moradia_vulneravel, axis=1)

# Remover as colunas que não serão mais necessárias
colunas_para_remover = ['A001', 'A002010', 'A003010', 'A004010', 'A01001', 'A011']
df.drop(columns=colunas_para_remover, inplace=True)

# Exibir as primeiras linhas para verificar a criação da nova coluna
print(df['moradia_vulneravel'].head())


0    2
1    2
2    2
3    2
4    2
Name: moradia_vulneravel, dtype: object


In [75]:
df['moradia_vulneravel'].isna().sum()

0

In [76]:
df['moradia_vulneravel'].values

array(['2', '2', '2', ..., '2', '2', '2'], dtype=object)

In [77]:
df['P03702']

0        30.0
1         NaN
2         0.0
3         0.0
4         NaN
         ... 
40354     NaN
40355     NaN
40356     NaN
40357     NaN
40358     0.0
Name: P03702, Length: 40359, dtype: float64

### Consolidando colunas relacionadas a prática de execícios físicos  

In [78]:


df['tempo_total_exercicio'] = df['P03701'] + df['P03702'] / 60  # Calcular o tempo total de exercício em horas
df['frequencia_exercicio'] = df['P035']  # Frequência de exercícios por semana

# Remover as colunas originais que não serão mais necessárias
colunas_para_remover = ['P03701', 'P03702', 'P035']
df.drop(columns=colunas_para_remover, inplace=True)

df[['tempo_total_exercicio', 'frequencia_exercicio']].head()


,tempo_total_exercicio,frequencia_exercicio
0,0.5,1.0
1,NaN,NaN
2,1.0,3.0
3,2.0,7.0
4,NaN,0.0


In [79]:
df[['tempo_total_exercicio', 'frequencia_exercicio']].isna().sum()

tempo_total_exercicio    25925
frequencia_exercicio     25445
dtype: int64

### Consolidando colunas relacionadas ao consumo de bebidas alcolicas

In [80]:
# Métricas relacionadas ao consumo de bebidas alcoólicas
df['doses_bebida_alcoolica'] = df['P02801'] * df['P029']  # Calcular o total de doses de bebidas alcoólicas por semana
df['freq_bebida_alcoolica'] = df['P027']  # Frequência de consumo de bebidas alcoólicas

# Remover as colunas originais que não serão mais necessárias
colunas_para_remover = ['P02801', 'P029', 'P027']
df.drop(columns=colunas_para_remover, inplace=True)

df[['doses_bebida_alcoolica', 'freq_bebida_alcoolica']].head(10)

,doses_bebida_alcoolica,freq_bebida_alcoolica
0,NaN,2.0
1,7.0,3.0
2,NaN,1.0
3,NaN,1.0
4,NaN,1.0
5,NaN,1.0
6,NaN,2.0
7,NaN,1.0
8,NaN,2.0
9,4.0,3.0


In [81]:
df[['doses_bebida_alcoolica', 'freq_bebida_alcoolica']].isna().sum()

doses_bebida_alcoolica    29376
freq_bebida_alcoolica         0
dtype: int64

### Removendo colunas irrelevantes

In [82]:
# Lista das colunas a serem removidas
colunas_para_remover = ['E01801', 'E01803', 'E01805', 'E01201', 'F001011', 'F007011', 'F008011', 'VDF001', 
                        'Q09301', 'Q094', 'Q09502', 'Q09605', 'Q09606', 'Q09607', 'Q098', 'Q100', 
                        'Q10101', 'Q10202', 'Q104', 'Q105', 'Q106', 'Q10701', 'Q109','E01604', 'E01603',
                        'P02601', 'P02102', 'P02101'  ]

# Remover as colunas especificadas
df.drop(columns=colunas_para_remover, inplace=True)

> Verificando se as colunas foram removidas

In [83]:
df.columns

Index(['C006', 'M01401', 'M01501', 'E01401', 'E014011', 'M005011', 'M00601',
       'D00901', 'Q092', 'A005010', 'N00101', 'P034', 'P036', 'J00402',
       'J01101', 'J014', 'J01502', 'H024', 'N010', 'P00601', 'P00602',
       'P00603', 'P00604', 'P00605', 'P00607', 'P00608', 'P00609', 'P00610',
       'P00611', 'P00612', 'P00613', 'P00614', 'P00615', 'P00616', 'P00617',
       'P00618', 'P00619', 'P00620', 'P00621', 'P00622', 'P00623', 'P006',
       'P00901', 'P01001', 'P01101', 'P013', 'P015', 'P02001', 'P01601',
       'P018', 'P019', 'P02002', 'P023', 'P02401', 'P02501', 'P02602',
       'peso_consolidado', 'altura_consolidada', 'renda_total', 'idade',
       'trabalhou', 'horas_trabalhadas_total', 'doencas_cronicas',
       'saneamento_basico', 'moradia_vulneravel', 'tempo_total_exercicio',
       'frequencia_exercicio', 'doses_bebida_alcoolica',
       'freq_bebida_alcoolica'],
      dtype='object')

## Tratando Valores nulos restantes

### Renomeando Colunas
> Renomeando nossas colunas para facilitar o entendimento

In [84]:
df.rename(columns={
    'C006': 'Sexo',
    'M01401': 'Apoio_Familiar',
    'M01501': 'Apoio_de_Amigos',
    'E01401': 'Tipo_de_Trabalho',
    'E014011': 'Trabalhador_Nao_Remunerado',
    'M005011': 'Horas_Trabalhadas_Noite',
    'M00601': 'Freq_Trabalho_Noite',
    'D00901': 'Curso_Mais_Elevado',
    'Q092': 'Diagnostico_Depressao',
    'N00101': 'Estado_de_Saude',
    'P034': 'Pratica_Exercicio',
    'P036': 'Exercicio_Mais_Frequente',
    'J00402': 'Motivo_Impedimento_Atividades',
    'J01101': 'Ultima_Consulta_Medica',
    'J014': 'Procura_Atendimento_Saude',
    'J01502': 'Motivo_Atendimento_Saude',
    'H024': 'Orientacoes_Saude',
    'N010': 'Problemas_Sono',
    'P00601': 'Consumo_Arroz_Macarrao',
    'P00602': 'Consumo_Batata',
    'P00603': 'Consumo_Feijao',
    'P00604': 'Consumo_Carne',
    'P00605': 'Consumo_Ovo',
    'P00607': 'Consumo_Verduras_1',
    'P00608': 'Consumo_Verduras_2',
    'P00609': 'Consumo_Verduras_3',
    'P00610': 'Consumo_Frutas_1',
    'P00611': 'Consumo_Frutas_2',
    'P00612': 'Consumo_Leite',
    'P00613': 'Consumo_Castanhas',
    'P00614': 'Consumo_Refrigerante',
    'P00615': 'Consumo_Suco_Caixinha',
    'P00616': 'Consumo_Bebida_Achocolatada',
    'P00617': 'Consumo_Salgadinho_Biscoito',
    'P00618': 'Consumo_Biscoito_Doce',
    'P00619': 'Consumo_Sobremesa',
    'P00620': 'Consumo_Embutidos',
    'P00621': 'Consumo_Pao',
    'P00622': 'Consumo_Molhos_Industrializados',
    'P00623': 'Consumo_Alimentos_Prontos',
    'P006': 'Freq_Consumo_Feijao',
    'P00901': 'Freq_Consumo_Verduras',
    'P01001': 'Tipo_Verdura_Costuma_Comer',
    'P01101': 'Freq_Consumo_Carne_Vermelha',
    'P013': 'Freq_Consumo_Frango',
    'P015': 'Freq_Consumo_Peixe',
    'P02001': 'Freq_Consumo_Suco_Caixinha',
    'P01601': 'Freq_Consumo_Suco_Natural',
    'P018': 'Freq_Consumo_Frutas',
    'P019': 'Freq_Comer_Frutas_Dia',
    'P02002': 'Freq_Consumo_Refrigerante',
    'P023': 'Freq_Consumo_Leite',
    'P02401': 'Tipo_Leite',
    'P02501': 'Freq_Consumo_Doces',
    'P02602': 'Freq_Substituicao_Refeicao',
   
}, inplace=True)

# Exibir as primeiras linhas para verificar as colunas renomeadas
df.head()


,Sexo,Apoio_Familiar,Apoio_de_Amigos,Tipo_de_Trabalho,Trabalhador_Nao_Remunerado,Horas_Trabalhadas_Noite,Freq_Trabalho_Noite,Curso_Mais_Elevado,Diagnostico_Depressao,A005010,...,idade,trabalhou,horas_trabalhadas_total,doencas_cronicas,saneamento_basico,moradia_vulneravel,tempo_total_exercicio,frequencia_exercicio,doses_bebida_alcoolica,freq_bebida_alcoolica
0,2.0,3.0,0.0,6.0,NaN,NaN,NaN,5.0,1.0,1.0,...,55.0,1.0,40.0,1,2,2,0.5,1.0,NaN,2.0
1,2.0,1.0,1.0,3.0,NaN,NaN,NaN,10.0,2.0,1.0,...,45.0,1.0,36.0,2,2,2,NaN,NaN,7.0,3.0
2,2.0,1.0,0.0,3.0,NaN,NaN,NaN,4.0,2.0,1.0,...,58.0,1.0,6.0,2,2,2,1.0,3.0,NaN,1.0
3,2.0,3.0,0.0,3.0,NaN,NaN,NaN,10.0,2.0,2.0,...,41.0,1.0,16.0,2,2,2,2.0,7.0,NaN,1.0
4,2.0,2.0,1.0,6.0,NaN,NaN,NaN,7.0,2.0,3.0,...,52.0,1.0,48.0,1,2,2,NaN,0.0,NaN,1.0


In [85]:
# Vericando o tamanho da nossa base 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40359 entries, 0 to 40358
Data columns (total 69 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   Sexo                             40359 non-null  category
 1   Apoio_Familiar                   40359 non-null  float64 
 2   Apoio_de_Amigos                  40359 non-null  float64 
 3   Tipo_de_Trabalho                 26285 non-null  category
 4   Trabalhador_Nao_Remunerado       295 non-null    category
 5   Horas_Trabalhadas_Noite          3145 non-null   float64 
 6   Freq_Trabalho_Noite              3145 non-null   category
 7   Curso_Mais_Elevado               36596 non-null  category
 8   Diagnostico_Depressao            40359 non-null  category
 9   A005010                          40359 non-null  category
 10  Estado_de_Saude                  40359 non-null  category
 11  Pratica_Exercicio                40359 non-null  category
 12  Exer

In [86]:
# Contar valores nulos em cada coluna
valores_nulos = df.isnull().sum()

# Filtrar colunas com valores nulos maiores que zero
colunas_com_valores_nulos = valores_nulos[valores_nulos > 0]

# Exibir colunas com valores nulos maiores que zero
print("Colunas com valores nulos:")
print(colunas_com_valores_nulos)


Colunas com valores nulos:
Tipo_de_Trabalho                 14074
Trabalhador_Nao_Remunerado       40064
Horas_Trabalhadas_Noite          37214
Freq_Trabalho_Noite              37214
Curso_Mais_Elevado                3763
Exercicio_Mais_Frequente         25925
Motivo_Impedimento_Atividades    35861
Motivo_Atendimento_Saude         31181
Orientacoes_Saude                35579
Tipo_Verdura_Costuma_Comer       18197
Freq_Comer_Frutas_Dia            21402
Tipo_Leite                       12653
tempo_total_exercicio            25925
frequencia_exercicio             25445
doses_bebida_alcoolica           29376
dtype: int64


> Para o tratamendo dos valores nulos teremos um notebook dedicado, onde iremos: 
>  - Identificar novamente nossas colunas com valores nulos.
>  - Remover colunas que não seram possiveis preencher estes valores. 
>  - Testar diferentes extatégias para preenchimento. 

In [87]:
# Exportando a base de dados 
df.to_csv(r"C:\Users\maype\OneDrive\Área de Trabalho\projects\projeto-mineracao-de-dados\Data\base_tratada.csv", index= False)